# Family example

This jupyter notebook presents some functionalities of the andante package through the family example available at 'andante/Examples/family.pl'

## Imports


In [1]:
from andante.program import AndanteProgram

## family.pl
The following cell outputs the contents of family.pl

In [2]:
# Prints the contents of "Examples/family.pl"
with open("Examples/family.pl") as file:
    print(file.read())

set(verbose,0).

modeh(1,parent(+person,-person)).
modeh(1,grandfather(+person,-person)).
modeh(1,grandparent(+person,-person)).
modeb(*,father(+person,-person)).
modeb(*,mother(+person,-person)).
modeb(*,parent(+person,-person)).
determination(grandfather/2,father/2).
determination(grandfather/2,mother/2).
determination(grandfather/2,parent/2).
determination(grandparent/2,father/2).
determination(grandparent/2,mother/2).
determination(grandparent/2,parent/2).
determination(parent/2,father/2).
determination(parent/2,mother/2).


%%%%%%%%%%%%%%%%%%%%%%
% Background knowledge

:- begin_bg.

person(andrew).  person(bernard).  person(cathleen).  person(daphne).
person(edith).  person(fred).  person(george).  person(john).
person(louis).  person(oscar).  person(paul).  person(robert).
person(stephen).  person(sylvia).  person(william). person(ada).

father(william,sylvia).
father(oscar,louis).
father(oscar,daphne).
father(oscar,cathleen).
father(oscar,fred).
father(oscar,bernard).
father(lo

## Parse family.pl

In [3]:
ap = AndanteProgram.build_from("Examples/family.pl")

## Family background knowledge in an image
<img src="andante/Examples/family_image.png" alt="Background knowledge in a picture" style="width: 600px;"/>

## Querying the background knowledge

In [4]:
from andante.parser import Parser
parser = Parser()
text = """
:- begin_bg.
    mortal(X):-man(X).
    man(socrates).
:- end_bg.
"""
knowledge = parser.parse(text, rule='background')
print(knowledge)

Knowledge object (class: TreeShapedKnowledge)
Clauses:
   mortal(X) :- man(X).
   man(socrates).


In [5]:
from andante.solver import AndanteSolver
solver = AndanteSolver()
q = parser.parse("mortal(X).", rule="query")

# Using method succeeds_on
success = solver.succeeds_on(q, knowledge)
if success:
    print("The query succeeded")
else:    
    print("The query failed")
    
# Using method query
print("The solutions for the query are:")
for solution in solver.query(q, knowledge):
    print('   ', solution)

The query succeeded
The solutions for the query are:
    {X: socrates}


### The metrics are as follow:
- p: the number of positive examples covered by the clause
- n: the number of negative examples covered by the clause
- c: the number of atom in the body of clause
- h: an optimistic guess to the number of atoms still needed in the clause 
- g: p-c-h
- f: g-n = p-(c+h+n)

In [9]:
text = """
% Mode declarations
modeh(*,daughter(+person,-person)).
modeb(*,parent(+person,-person)).
modeb(*,parent(-person,+person)).
modeb(*,female(+person)).
modeb(*,female(-person)).

% Determinations
determination(daughter/2,parent/2).
determination(daughter/2,female/1).

% Background knowledge
:- begin_bg.
person(ann). person(mary). person(tom). person(eve). person(lucy).
female(ann). female(mary).              female(eve). female(lucy).
parent(ann,mary). 
parent(ann,tom). 
parent(tom,eve). 
parent(tom,lucy).
:- end_bg.
    
% Positive examples
:- begin_in_pos.
daughter(lucy,tom).
daughter(mary,ann).
daughter(eve,tom).
:- end_in_pos.

% Negative examples
:- begin_in_neg.
daughter(tom,ann).
daughter(tom,eve).
daughter(ann,tom).
:- end_in_neg.
"""
pr = parser.parse(text, rule="andantefile")
H = pr.induce(verbose=1)

print("Knowledge learned")
print(H)

Examples: 3 positives - 3 negatives
Current example: daughter(lucy, tom).
Bottom_i: daughter(A, B) :- parent(B, A), female(A), female(C), female(D), female(E), parent(E, B), parent(B, D), parent(E, C).
Metrics: [  p,  n,  c,  h,  g,  f]
Candidate: [  3,  3,  0,  1,  2, -1] daughter(A, B).
Candidate: [  3,  1,  1,  1,  1,  0] daughter(A, B) :- parent(B, A).
Candidate: [  3,  1,  1,  1,  1,  0] daughter(A, B) :- parent(B, A).
Candidate: [  3,  0,  2,  0,  1,  1] daughter(A, B) :- parent(B, A), female(A).
Candidate: [  3,  1,  1,  1,  1,  0] daughter(A, B) :- parent(B, A).
Candidate: [  3,  1,  2,  1,  0, -1] daughter(A, B) :- parent(B, A), female(C).
Candidate: [  3,  1,  1,  1,  1,  0] daughter(A, B) :- parent(B, A).
Candidate: [  3,  1,  2,  1,  0, -1] daughter(A, B) :- parent(B, A), female(D).
Candidate: [  3,  1,  1,  1,  1,  0] daughter(A, B) :- parent(B, A).
Candidate: [  3,  1,  2,  1,  0, -1] daughter(A, B) :- parent(B, A), female(E).
Candidate: [  3,  1,  1,  1,  1,  0] daughter

In [7]:
text = """
:- begin_bg.

mortal(X):-man(X).
man(socrates).

:- end_bg.
"""
ap2 = AndanteProgram.build_from(text)

In [8]:
ap2.query('mortal(X).')

(True,
           0
 X  socrates)

In [9]:
#ap.set('verbose',1) # Uncomment to see the details

# Is ada the mother of sylvia ?
print(ap.query('mother(ada, sylvia).'))

# Of whom is oscar the father ?
print(ap.query('father(oscar, X).'))

(True, Empty DataFrame
Columns: [0]
Index: [])
(True,         0     1      2        3         4
X  daphne  fred  louis  bernard  cathleen)


## Inductive learning of the examples

In [10]:
ap.induce(update_knowledge=True, logging=True, verbose=1)

Examples: 66p/100n
Current example: parent(ada, sylvia).
Bottom_i: parent(A, B) :- mother(A, B), mother(B, C), mother(B, D), mother(B, E), mother(B, F).
parent(A, B).: [  0,  1, 22, 48, 21,-27] parent(A, B).
parent(A, B) :- mother(A, B).: [  1,  0, 10,  0,  9,  9] parent(A, B) :- mother(A, B).
parent(A, B).: [  0,  1, 22, 48, 21,-27] parent(A, B).
parent(A, B).: [  0,  1, 22, 48, 21,-27] parent(A, B).
parent(A, B).: [  0,  1, 22, 48, 21,-27] parent(A, B).
parent(A, B).: [  0,  1, 22, 48, 21,-27] parent(A, B).
Clause: parent(A, B) :- mother(A, B).

Examples: 56p/100n
Current example: parent(george, oscar).
Bottom_i: parent(A, B) :- father(A, B), father(B, C), father(B, D), father(B, E), father(B, F), father(B, G).
parent(A, B).: [  0,  1, 12, 48, 11,-37] parent(A, B).
parent(A, B) :- father(A, B).: [  1,  0, 12,  0, 11, 11] parent(A, B) :- father(A, B).
Clause: parent(A, B) :- father(A, B).

Examples: 44p/100n
Current example: grandfather(george, bernard).
Bottom_i: grandfather(A, B) :-

Knowledge object (class: TreeShapedKnowledge)
Clauses:
   parent(A, B) :- mother(A, B).
   parent(A, B) :- father(A, B).
   grandfather(A, B) :- father(A, C), parent(C, B).
   grandparent(A, B) :- parent(A, C), parent(C, B).

## Verifying if a clause is implicitly present in the theory

In [11]:
ap.verify('grandparent(A,B):-grandfather(A,B).', verbose=0)

(True,
 Empty DataFrame
 Columns: [0]
 Index: [])

In [12]:
ap.verify('grandfather(A,B):-grandparent(A,B),mother(A,C).', verbose=0)

(False, [])